Latest update: 2022-11-14 10:49:31.110292

<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorial #13: Download Zoo classifications created by students</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>

# Set up and requirements

## Import Python packages

In [ ]:
# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Set the directory of the libraries
import sys

sys.path.append("..")

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.t8_utils as t8

print("Packages loaded successfully")

## Choose your project

In [ ]:
project_name = t_utils.choose_project()

## Initiate the database

In [ ]:
project = p_utils.find_project(project_name=project_name.value)
# Initiate db
db_info_dict = t_utils.initiate_db(project)
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project)

# Retrieve the student classifications

## Retrieve all zooniverse classifications

In [ ]:
zoo_info_dict = t_utils.retrieve__populate_zoo_info(
    project=project,
    db_info_dict=db_info_dict,
    zoo_project=zoo_project,
    zoo_info=["classifications"],
)

## Filter and download zooniverse classifications

In [ ]:
# base imports
import os
import pandas as pd
import logging

# widget imports
import ipywidgets as widgets
from ipywidgets import interactive, Layout, VBox
from IPython.display import display
import asyncio


logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
out_df = pd.DataFrame()


def filter_classifications(zoo_info_dict: dict):
    """
    This function allows the user to specify a username prefix to download only their zoo classifications

    :param zoo_info_dict: a dictionary with the following keys:
    :type zoo_info_dict: dict
    :return: A text widget and a click button to download classifications.
    """
    # Load the dic as a pandas df
    zoo_classifications = zoo_info_dict["classifications"]

    # Create the text widget to specify the prefix of the users
    prefix_i = widgets.Text(
        placeholder="",
        description="Prefix of the usernames:",
        disabled=False,
        layout=Layout(width="95%"),
        style={"description_width": "initial"},
    )

    # Create the download classifications button
    download_button = widgets.Button(
        description="Download classifications",
        layout=Layout(width="25%"),
        style={"description_width": "initial"},
        button_style="danger",
    )

    # When button is clicked filter classifications with that prefix and download classifications in a csv
    async def f(prefix_i):
        x = await t_utils.wait_for_change(
            download_button
        )  # <---- Pass buttons into the function
        if x == "Download classifications":
            print("Your classifications will be downloaded shortly.")

            # Filter classifications with the prefix of interest
            zoo_classifications_i = zoo_classifications[
                zoo_classifications["Username"].str.startswith(prefix_i.result)
            ].reset_index(drop=True)

            # Create a csv with the classifications
        #             zoo_classifications_i.to_csv("classifications_zoo.csv", index=False)

        else:
            print("There was an issue downloading your classifications!")

    print("")
    print("")
    print("Download student classifications")
    display(
        VBox([prefix_widget, download_button])
    )  # <----Display both widgets in an Vbox
    asyncio.create_task(f(prefix_i))

In [ ]:
filter_classifications(zoo_info_dict)

# Check subjects based on their id

In [ ]:
# Launch the subject viewer
t8.launch_viewer(agg_class_df, wm.checks["Subject type: #0"])

In [ ]:
# END